# Self-definded potentials

**`By You-Liang Zhu`**

Here, we take tetramer molecule as an example system to illustrate the usage of self-defined potential functions for simulation. The potentials could include non-bonded, bond, angle, and dihedral ones.

First of all, we generate the configuration of a tetramer system by molgen module.

In [ ]:
import molgen

mol1=molgen.Molecule(4)#particle number
mol1.setParticleTypes("A,A,A,A")#type
mol1.setTopology("0-1,1-2,2-3")#topology
mol1.setBondLength(0.5)#bond length
mol1.setAngleDegree("A","A","A",120)#bond length
mol1.setDihedralDegree("A","A","A","A",180)#bond length
mol1.setMass(1.0)#mass


gen=molgen.Generators(20,20,20) # box size in x, y, and z direction
gen.addMolecule(mol1,1)#molecule, the number of molecules
gen.outPutMST("example") #file name

Then we could run MD simulation by calling the potential functions from the library of pygamd module by giving the function names.

In [ ]:
import pygamd

mst = pygamd.snapshot.read("example.mst")
app = pygamd.application.dynamics(info=mst, dt=0.001)

fn = pygamd.force.nonbonded(info=mst, rcut=3.0, func='lj')
fn.setParams(type_i="A", type_j="A", param=[1.0, 1.0, 1.0, 3.0])
app.add(fn)

fb = pygamd.force.bond(info=mst, func='harmonic')
fb.setParams(bond_type = 'A-A', param=[1000.0, 0.5])#(,k, r0)
app.add(fb)

fa = pygamd.force.angle(info=mst, func='harmonic')
fa.setParams(angle_type='A-A-A', param=[100.0, 120.0])#(,k, t0)
app.add(fa)

 
fd = pygamd.force.dihedral(info=mst, func='harmonic')
fd.setParams(dihedral_type = 'A-A-A-A', param=[25.0, 180.0])#(,k, t0)
app.add(fd)

nvt = pygamd.integration.nvt(info=mst, group='all', method="nh", tau=0.5, temperature=1.0)
app.add(nvt)

dd = pygamd.dump.data(info=mst, group='all', file='data.log', period=100)
app.add(dd)

dm = pygamd.dump.mst(info=mst, group='all', file='p.mst', period=10000)
app.add(dm)

#run 
app.run(20000)

Instead of calling the potential functions from library, we could self-defined potential functions for non-bonded, bond, angle, and dihedral interactions in script. The self-defined functions are device function in Python3 Numba lauguages. The illustration about the expressions of potential and force in self-definded functions could be found in https://pygamd-v1.readthedocs.io/en/latest/. 

In [ ]:
import pygamd
from numba import cuda
import numba as nb
import math


mst = pygamd.snapshot.read("example.mst")
app = pygamd.application.dynamics(info=mst, dt=0.001)

@cuda.jit(device=True)
def lj(rsq, param, fp):
	epsilon = param[0]
	sigma = param[1]
	alpha = param[2]
	rcut = param[3]
	if rsq<rcut*rcut:
		sigma2 = sigma*sigma
		r2inv = sigma2/rsq;
		r6inv = r2inv * r2inv * r2inv;
		f = nb.float32(4.0) * epsilon * r2inv * r6inv * (nb.float32(12.0) * r6inv - nb.float32(6.0) * alpha)/sigma2	
		p = nb.float32(4.0) * epsilon * r6inv * ( r6inv - nb.float32(1.0))
		fp[0]=f
		fp[1]=p	

fn = pygamd.force.nonbonded(info=mst, rcut=3.0, func=lj)
fn.setParams(type_i="A", type_j="A", param=[1.0, 1.0, 1.0, 3.0])
app.add(fn)

@cuda.jit(device=True)
def bond_harmonic(rsq, param, fp):
	k = param[0]
	r0 = param[1]
	r = math.sqrt(rsq)
	f = k * (r0/r - nb.float32(1.0))
	p = nb.float32(0.5) * k * (r0 - r) * (r0 - r)
	fp[0]=f
	fp[1]=p

fb = pygamd.force.bond(info=mst, func=bond_harmonic)
fb.setParams(bond_type = 'A-A', param=[1000.0, 0.5])#param=[k, r0]
app.add(fb)


@cuda.jit(device=True)
def angle_harmonic(cos_abc, sin_abc, param, fp):
	k = param[0]
	t0 = param[1]
	dth = math.acos(cos_abc) - math.pi*t0/nb.float32(180.0)
	f = k * dth
	p = nb.float32(0.5) * f * dth
	fp[0]=f
	fp[1]=p

fa = pygamd.force.angle(info=mst, func=angle_harmonic)
fa.setParams(angle_type='A-A-A', param=[100.0, 120.0])#param=[k, t0]
app.add(fa)

@cuda.jit(device=True)
def dihedral_harmonic(cos_abcd, sin_abcd, param, fp):
	k = param[0]
	cos_phi0 = param[1]
	sin_phi0 = param[2]
	cos_factor = param[3]
	f = cos_factor * (-sin_abcd*cos_phi0 + cos_abcd*sin_phi0)
	p = nb.float32(1.0) + cos_factor * (cos_abcd*cos_phi0 + sin_abcd*sin_phi0)
	fp[0]=-k*f
	fp[1]=k*p
	
fd = pygamd.force.dihedral(info=mst, func=dihedral_harmonic)
fd.setParams(dihedral_type = 'A-A-A-A', param=[25.0, math.cos(math.pi), math.sin(math.pi), -1.0])#param=[k, cos_phi0, sin_phi0, cos_factor]
app.add(fd)

nvt = pygamd.integration.nvt(info=mst, group='all', method="nh", tau=0.5, temperature=1.0)
app.add(nvt)

dd = pygamd.dump.data(info=mst, group='all', file='data.log', period=100)
app.add(dd)

dm = pygamd.dump.mst(info=mst, group='all', file='p.mst', period=10000)
app.add(dm)

#run 
app.run(20000)